In [1]:
# import dependencies
import pandas as pd

In [2]:
# import files 
gdp_df = pd.read_csv("Data Sources/GDP by Country by Year (The World Bank).csv")

tourism_gdp = pd.read_excel("Data Sources/Tourism GDP as a Percent of Total GDP by Country by Year (UNWTO).xlsx")

In [18]:
# get a list of column headers for reference
print(gdp_df.columns)

print(tourism_gdp.columns)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')
Index(['Indicator', 'SeriesID', 'SeriesDescription', 'GeoAreaCode',
       'GeoAreaName', 'Country Code', 'TimePeriod', 'Total', 'Time_Detail',
       'Source', 'FootNote', 'Nature', 'Units', 'Reporting Type',
       'SeriesCode'],
      dtype='object')


In [ ]:
# Remove Years from our dataset

# columns that we need to keep
cols_to_keep = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
        '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# new dataframe
gdp_keep_df = gdp_df[cols_to_keep]

In [ ]:
# Transform the data from wide to long
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']

cols_to_melt = ['2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


gdp_long_df = pd.melt(gdp_keep_df, id_vars=id_vars, value_vars=cols_to_melt, var_name='Year',value_name='GDP')

In [21]:
# set time period to string for merge
tourism_gdp['TimePeriod'] = tourism_gdp['TimePeriod'].astype(str)
tourism_gdp.dtypes


Indicator             object
SeriesID               int64
SeriesDescription     object
GeoAreaCode            int64
GeoAreaName           object
Country Code          object
TimePeriod            object
Total                float64
Time_Detail            int64
Source                object
FootNote              object
Nature                object
Units                 object
Reporting Type        object
SeriesCode            object
dtype: object

In [ ]:
merge_df = pd.merge(tourism_gdp,gdp_long_df,how='inner', left_on=['Country Code','TimePeriod'],right_on=['Country Code','Year'])

In [23]:
merge_df.to_csv('Cleaned Data/Merged GDP and Tourism data.csv')